<a href="https://colab.research.google.com/github/DorcasOJ/hamoye-internship/blob/main/hamoye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Datasets/Hamoye_internship/

/content/drive/MyDrive/Datasets/Hamoye_internship


In [ ]:
%ls

energydata_complete.csv


In [49]:
import pandas as pd, numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [36]:
df = pd.read_csv('energydata_complete.csv')
data = df.copy()

In [37]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [38]:
#df_date = pd.to_datetime(df['date'])
#df['year'] = df_date.dt.year
#df['month'] = df_date.dt.month
#df['day'] = df_date.dt.day
#df['hour'] = df_date.dt.hour
#df['minute'] = df_date.dt.minute

In [39]:
df.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,19.592106,50.949283,7.910939,54.609083,20.267106,35.388200,22.029107,42.936165,19.485828,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,1.844623,9.022034,6.090347,31.149806,2.109993,5.114208,1.956162,5.224361,2.014712,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,15.330000,29.815000,-6.065000,1.000000,15.390000,23.200000,16.306667,29.600000,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,18.277500,45.400000,3.626667,30.025000,18.700000,31.500000,20.790000,39.066667,18.000000,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,19.390000,49.090000,7.300000,55.290000,20.033333,34.863333,22.100000,42.375000,19.390000,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,20.619643,53.663333,11.256000,83.226667,21.600000,39.000000,23.390000,46.536000,20.600000,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,25.795000,96.321667,28.290000,99.900000,26.000000,51.400000,27.230000,58.780000,24.500000,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530


In [40]:
# independent and dependent variable
target = df['Appliances']
df = df.drop(columns=['date', 'lights', 'Appliances'])

In [41]:
# normalise using minmax scaling
scale = MinMaxScaler()
norm_df = pd.DataFrame(scale.fit_transform(df), columns= df.columns)

In [42]:
# normalised data
norm_df.head()

,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,0.32735,0.566187,0.225345,0.684038,0.215188,0.746066,0.351351,0.764262,0.175506,0.381691,0.381070,0.841827,0.170594,0.653428,0.173329,0.661412,0.223032,0.677290,0.372990,0.097674,0.894737,0.500000,0.953846,0.538462,0.265449,0.265449
1,0.32735,0.541326,0.225345,0.682140,0.215188,0.748871,0.351351,0.782437,0.175506,0.381691,0.375443,0.839872,0.170594,0.651064,0.173329,0.660155,0.226500,0.678532,0.369239,0.100000,0.894737,0.476190,0.894872,0.533937,0.372083,0.372083
2,0.32735,0.530502,0.225345,0.679445,0.215188,0.755569,0.344745,0.778062,0.175506,0.380037,0.367487,0.830704,0.170594,0.646572,0.173329,0.655586,0.219563,0.676049,0.365488,0.102326,0.894737,0.452381,0.835897,0.529412,0.572848,0.572848
3,0.32735,0.524080,0.225345,0.678414,0.215188,0.758685,0.341441,0.770949,0.175506,0.380037,0.363800,0.833401,0.164310,0.641489,0.164175,0.650788,0.219563,0.671909,0.361736,0.104651,0.894737,0.428571,0.776923,0.524887,0.908261,0.908261
4,0.32735,0.531419,0.225345,0.676727,0.215188,0.758685,0.341441,0.762697,0.178691,0.380037,0.361859,0.848264,0.170594,0.639362,0.164175,0.650788,0.219563,0.671909,0.357985,0.106977,0.894737,0.404762,0.717949,0.520362,0.201611,0.201611


In [43]:
xtrain, xtest, ytrain, ytest = train_test_split(norm_df, target, test_size=0.3, random_state= 42)

In [44]:
xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

((13814, 26), (13814,), (5921, 26), (5921,))

In [45]:
#modelling
model = LinearRegression()
model.fit(xtrain, ytrain)
predicted = model.predict(xtest)

In [46]:
# first model mean absolute error
mae = mean_absolute_error(ytest, predicted)

In [47]:
print(round(mae, 2))

53.64


In [48]:
# Residual sum of square
rss = np.sum(np.square(ytest, predicted))
print(round(rss, 2))

117425400.0


In [50]:
# root mean squared error
rmse = np.sqrt(mean_squared_error(ytest, predicted))
print(round(rmse, 2))

60919.5


In [51]:
#Coefficient of Determination
cod = r2_score(ytest, predicted)
print(round(cod, 2))

-360216.57


In [53]:
#  question 12 & 13

x = pd.DataFrame(data['T2'])
y = data['T6']
model2 = LinearRegression()
model2.fit(x, y)
pred = model2.predict(x)
r2 = r2_score(y, pred)
print(round(r2, 2))

0.64


In [67]:
# lasso
from sklearn.linear_model import Lasso
lasso_model = Lasso(alpha=0.001)
lasso_model.fit(xtrain, ytrain)
lasso_pred = lasso_model.predict(xtest)

In [68]:
# lasso root mean squared error
ridge_rmse = np.sqrt(mean_squared_error(ytest, lasso_pred))
print(round(rmse, 2))

60919.5


In [64]:
# ridge
from sklearn.linear_model import Ridge
ridge_model = Ridge(alpha=0.4)
ridge_model.fit(xtrain, ytrain)
ridge_pred = ridge_model.predict(xtest)

In [66]:
# ridge root mean squared error
ridge_rmse = np.sqrt(mean_squared_error(ytest, ridge_pred))
print(round(rmse, 2))

60919.5


In [57]:
# get weights
def get_w_df(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    w_df = pd.DataFrame(weights).reset_index()
    w_df.columns = ['features', col_name]
    w_df[col_name].round(3)
    return w_df

In [62]:

model_weights = get_w_df(model, xtrain, 'Linear Model Weight')
ridge_weights = get_w_df(ridge_model, xtrain, 'Ridge Model Weight')
lasso_weights = get_w_df(lasso_model, xtrain, 'Lasso Model Weight')


In [ ]:
# merging all weights
all_weights = pd.merge(model_weights, ridge_weights, on='features')
all_weights = pd.merge(all_weights, lasso_weights, on='features')

In [63]:
all_weights

,features,Linear Model Weight,Ridge Model Weight,Lasso Model Weight
0,RH_2,-488.666805,-439.846215,-484.565333
1,T_out,-344.389845,-280.524329,-332.571693
2,T2,-252.710373,-215.494498,-249.468798
3,T9,-203.236627,-202.140437,-202.502545
4,RH_8,-168.627161,-167.808155,-168.695476
5,RH_out,-83.107599,-58.554305,-78.000385
6,RH_7,-47.736592,-49.195346,-47.389094
7,RH_9,-42.586339,-44.262196,-42.455100
8,T5,-16.752822,-21.242949,-17.118723
9,T1,-3.510725,-19.694643,-4.147124
